Arushi Nigam

### Import Statements

In [1]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn import metrics
import matplotlib.pyplot as plt
import math
from sklearn.neighbors import DistanceMetric
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.tools.eval_measures import mse
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import statistics
from scipy.stats import bootstrap
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
import warnings
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import RocCurveDisplay
from imblearn.over_sampling import SMOTE 
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import LabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn import svm, datasets
from sklearn.preprocessing import label_binarize
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn import tree
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale 
from sklearn.model_selection import RepeatedKFold
from sklearn import model_selection
from sklearn import preprocessing
from cmath import nan
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import hamming_loss, silhouette_score
from sklearn.cluster import KMeans
import statistics
from statistics import mode
from scipy.spatial.distance import hamming

/Users/arushinigam/opt/anaconda3/envs/dsci552/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### 1a.

In [2]:
warnings.filterwarnings('ignore')

frogscomp = pd.read_csv("../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv")
stringlabels = frogscomp[['Family', 'Genus', 'Species']]

#Encode Labels
lencode = preprocessing.LabelEncoder()
frogscomp['Family'] = lencode.fit_transform(frogscomp['Family'])
frogscomp['Genus'] = lencode.fit_transform(frogscomp['Genus'])
frogscomp['Species'] = lencode.fit_transform(frogscomp['Species'])

#Split train and test set
frogscomp70 = frogscomp.sample(frac=0.7, random_state=0)
frogscomp30 = frogscomp.drop(frogscomp70.index) 

#Get train and test set
labelstrain = frogscomp70[['Family', 'Genus', 'Species']]
frogtrain = frogscomp70.drop(frogscomp70[['Family', 'Genus', 'Species', 'RecordID']], axis=1)
labelstest = frogscomp30[['Family', 'Genus', 'Species']]
frogtest = frogscomp30.drop(frogscomp30[['Family', 'Genus', 'Species', 'RecordID']], axis=1)

### 1b(ii).

In [3]:
parameters = {'C':[0.001,0.01,0.1,1], 'gamma': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}
model = GridSearchCV(SVC(), parameters, cv=10)

In [4]:
fm = model.fit(frogtrain, labelstrain['Family'])
print("Family SVM Penalty:", fm.best_params_['C'])
print("Family Gaussian Kernel width:", 1/np.sqrt(2*(fm.best_params_['gamma'])))
ypredictfm = fm.predict(frogtest)
print("Exact Match Score:", np.sum(np.equal(np.array(labelstest['Family']), ypredictfm))/float(np.array(labelstest['Family']).size))
print("Hamming Loss:", np.sum(np.not_equal(np.array(labelstest['Family']), ypredictfm))/float(np.array(labelstest['Family']).size))

Family SVM Penalty: 1
Family Gaussian Kernel width: 0.7071067811865475
Exact Match Score: 0.9879573876794813
Hamming Loss: 0.012042612320518759


In [5]:
gm = model.fit(frogtrain, labelstrain['Genus'])
print("Genus SVM Penalty:", gm.best_params_['C'])
print("Genus Gaussian Kernel width:", 1/np.sqrt(2*(gm.best_params_['gamma'])))
ypredictgm = gm.predict(frogtest)
print("Exact Match Score:", np.sum(np.equal(np.array(labelstest['Genus']), ypredictgm))/float(np.array(labelstest['Genus']).size))
print("Hamming Loss:", np.sum(np.not_equal(np.array(labelstest['Genus']), ypredictgm))/float(np.array(labelstest['Genus']).size))

Genus SVM Penalty: 1
Genus Gaussian Kernel width: 0.7071067811865475
Exact Match Score: 0.9856415006947661
Hamming Loss: 0.014358499305233904


In [6]:
sm = model.fit(frogtrain, labelstrain['Species'])
print("Species SVM Penalty:", sm.best_params_['C'])
print("Species Gaussian Kernel width:", 1/np.sqrt(2*(sm.best_params_['gamma'])))
ypredictsm = sm.predict(frogtest)
print("Exact Match Score:", np.sum(np.equal(np.array(labelstest['Species']), ypredictsm))/float(np.array(labelstest['Species']).size))
print("Hamming Loss:", np.sum(np.not_equal(np.array(labelstest['Species']), ypredictsm))/float(np.array(labelstest['Species']).size))

Species SVM Penalty: 1
Species Gaussian Kernel width: 0.7453559924999299
Exact Match Score: 0.9856415006947661
Hamming Loss: 0.014358499305233904


### 1b(iii).

In [7]:
parameters = {'C':[0.001,0.01,0.1,1]}
model2 = GridSearchCV(LinearSVC(penalty='l1', loss='squared_hinge', dual=False), parameters, cv=10)

scale = StandardScaler()
frogtrainscaled = scale.fit_transform(frogtrain)
frogtestscaled = scale.fit_transform(frogtest)

In [8]:
fml = model2.fit(frogtrainscaled, labelstrain['Family'])
print("Family SVM Penalty:", fml.best_params_['C'])
ypredictfml = fml.predict(frogtestscaled)
print("Exact Match Score:", np.sum(np.equal(np.array(labelstest['Family']), ypredictfml))/float(np.array(labelstest['Family']).size))
print("Hamming Loss:", np.sum(np.not_equal(np.array(labelstest['Family']), ypredictfml))/float(np.array(labelstest['Family']).size))

Family SVM Penalty: 0.1
Exact Match Score: 0.9434923575729505
Hamming Loss: 0.05650764242704956


In [9]:
gml = model2.fit(frogtrainscaled, labelstrain['Genus'])
print("Genus SVM Penalty:", gml.best_params_['C'])
ypredictgml = gml.predict(frogtestscaled)
print("Exact Match Score:", np.sum(np.equal(np.array(labelstest['Genus']), ypredictgml))/float(np.array(labelstest['Genus']).size))
print("Hamming Loss:", np.sum(np.not_equal(np.array(labelstest['Genus']), ypredictgml))/float(np.array(labelstest['Genus']).size))

Genus SVM Penalty: 1
Exact Match Score: 0.9546086150995832
Hamming Loss: 0.04539138490041686


In [10]:
sml = model2.fit(frogtrainscaled, labelstrain['Species'])
print("Species SVM Penalty:", sml.best_params_['C'])
ypredictsml = sml.predict(frogtestscaled)
print("Exact Match Score:", np.sum(np.equal(np.array(labelstest['Species']), ypredictsml))/float(np.array(labelstest['Species']).size))
print("Hamming Loss:", np.sum(np.not_equal(np.array(labelstest['Species']), ypredictsml))/float(np.array(labelstest['Species']).size))

Species SVM Penalty: 1
Exact Match Score: 0.9583140342751274
Hamming Loss: 0.041685965724872626


### 1b(iv).

In [11]:
pline = imbpipeline(steps = [['SMOTE', SMOTE(random_state=0)],['model', LinearSVC(penalty='l1', loss='squared_hinge', dual=False)]])
parameters = {'model__C':[0.001,0.01,0.1,1]}
model3 = GridSearchCV(estimator=pline, param_grid=parameters, cv=10)

In [12]:
fmsm = model3.fit(frogtrain, labelstrain['Family'])
print("Family SVM Penalty:", fmsm.best_params_['model__C'])
ypredictfmsm = fmsm.predict(frogtest)
print("Exact Match Score:", np.sum(np.equal(np.array(labelstest['Family']), ypredictfmsm))/float(np.array(labelstest['Family']).size))
print("Hamming Loss:", np.sum(np.not_equal(np.array(labelstest['Family']), ypredictfmsm))/float(np.array(labelstest['Family']).size))

Family SVM Penalty: 1
Exact Match Score: 0.9254284390921723
Hamming Loss: 0.0745715609078277


In [13]:
gmsm = model3.fit(frogtrain, labelstrain['Genus'])
print("Genus SVM Penalty:", gmsm.best_params_['model__C'])
ypredictgmsm = gmsm.predict(frogtest)
print("Exact Match Score:", np.sum(np.equal(np.array(labelstest['Genus']), ypredictgmsm))/float(np.array(labelstest['Genus']).size))
print("Hamming Loss:", np.sum(np.not_equal(np.array(labelstest['Genus']), ypredictgmsm))/float(np.array(labelstest['Genus']).size))

Genus SVM Penalty: 1
Exact Match Score: 0.9272811486799444
Hamming Loss: 0.07271885132005558


In [14]:
smsm = model3.fit(frogtrain, labelstrain['Species'])
print("Species SVM Penalty:", smsm.best_params_['model__C'])
ypredictsmsm = smsm.predict(frogtest)
print("Exact Match Score:", np.sum(np.equal(np.array(labelstest['Species']), ypredictsmsm))/float(np.array(labelstest['Species']).size))
print("Hamming Loss:", np.sum(np.not_equal(np.array(labelstest['Species']), ypredictsmsm))/float(np.array(labelstest['Species']).size))

Species SVM Penalty: 1
Exact Match Score: 0.9559981472904122
Hamming Loss: 0.04400185270958777


After training all of the different classifiers, it appears that the SVM with cross-validation done on C and gamma did the best with the highest exact match score while the next best one was the L1-penalized. Surprisingly, the SMOTE one did not perform better. 

### 2(a,b, and c).

In [18]:
frogys = frogscomp[['Family', 'Genus', 'Species']]
frogxs = frogscomp.drop(frogscomp[['Family', 'Genus', 'Species', 'RecordID']], axis=1)
clusters = pd.DataFrame()
clusters["index"] = frogxs.index.values
avghdist = [];
avghscore = [];
avghloss = [];

for i in range(1,51):
    print(" ")
    print("Iteration:", i)
    silscores = [];
    # ------------ PART A --------------
    for k in range(1,51):
        if k == 1:
            silscores.append((k, 0))
        else:
            km = KMeans(n_clusters=k)
            km = km.fit(frogxs)
            clusterLabels = km.labels_
            silscore = silhouette_score(frogxs, clusterLabels)
            silscores.append((k, silscore))

    silscores.sort(key=lambda x: x[1], reverse=True)
    bestk = silscores[0][0]    
    print("Best k:", bestk)

    # ------------ PART B --------------
    kmb = KMeans(n_clusters=bestk)
    kmb = kmb.fit(frogxs)
    hi = kmb.labels_
    clusters['cluster_num'] = hi
    hsc = [];
    hls = [];
    hds = [];

    for j in range(0, bestk):
        families = [];
        genuses = [];
        speciess = [];   
        f = [];
        g = [];
        s = []; 
        clpoints = [];
        
        points = clusters[clusters.cluster_num == j]
        for index in points['index']:
            p = [];

            family = stringlabels.iloc[index]['Family']
            families.append(family)
            genus = stringlabels.iloc[index]['Genus']
            genuses.append(genus)
            species = stringlabels.iloc[index]['Species']
            speciess.append(species)

            fa = frogys.iloc[index]['Family']
            f.append(fa)
            ge = frogys.iloc[index]['Genus']
            g.append(ge)
            sp = frogys.iloc[index]['Species']
            s.append(sp)

            p.append(fa)
            p.append(ge)
            p.append(sp)
            clpoints.append(p)

        majf = mode(families) 
        majg = mode(genuses)
        majs = mode(speciess)
        mf = mode(f)
        mg = mode(g)
        ms = mode(s)
        print("Majority Label for cluster ", j, ": ", (majf, majg, majs))
        
        # ------------ PART C --------------
        truepoints = [];
        for j in range(len(families)):
            pt = [];
            pt.append(mf)
            pt.append(mg)
            pt.append(ms)
            truepoints.append(pt)
        
        hd = 0;
        hs = 0;
        for a,b in zip(truepoints, clpoints):
            if (a[0] != b[0]):
                hd = hd+1
            if (a[1] != b[1]):
                hd = hd+1
            if (a[2] != b[2]):
                hd = hd+1
            if (a[0] == b[0]):
                hs = hs+1
            if (a[1] == b[1]):
                hs = hs+1
            if (a[2] == b[2]):
                hs = hs+1
        
        hds.append(hd)
        hsc.append(hs)

    hamdist = sum(hds)
    hamscore = (sum(hsc)/(len(frogxs)*3))
    hamloss = 1 - hamscore

    print("Hamming Distance: ", hamdist)
    print("Hamming Score: ", hamscore)
    print("Hamming Loss: ", hamloss)
    
    avghscore.append(hamscore)
    avghloss.append(hamloss)
    avghdist.append(hamdist)
    
print("")
print("Average of all 50 Hamming Scores:", sum(avghscore)/len(avghscore))
print("Standard deviation of all 50 Hamming Scores:", statistics.stdev(avghscore))
print("")
print("Average of all 50 Hamming Losses:", sum(avghloss)/len(avghloss))
print("Standard deviation of all 50 Hamming Losses:", statistics.stdev(avghloss))
print("")
print("Average of all 50 Hamming Distances:", sum(avghdist)/len(avghdist))
print("Standard deviation of all 50 Hamming Distances:", statistics.stdev(avghdist))

 
Iteration: 1
Best k: 4
Majority Label for cluster  0 :  ('Hylidae', 'Hypsiboas', 'HypsiboasCordobae')
Majority Label for cluster  1 :  ('Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus')
Majority Label for cluster  2 :  ('Hylidae', 'Hypsiboas', 'HypsiboasCinerascens')
Majority Label for cluster  3 :  ('Dendrobatidae', 'Ameerega', 'Ameeregatrivittata')
Hamming Distance:  4801
Hamming Score:  0.7775770210794534
Hamming Loss:  0.22242297892054663
 
Iteration: 2
Best k: 4
Majority Label for cluster  0 :  ('Dendrobatidae', 'Ameerega', 'Ameeregatrivittata')
Majority Label for cluster  1 :  ('Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus')
Majority Label for cluster  2 :  ('Hylidae', 'Hypsiboas', 'HypsiboasCordobae')
Majority Label for cluster  3 :  ('Hylidae', 'Hypsiboas', 'HypsiboasCinerascens')
Hamming Distance:  4801
Hamming Score:  0.7775770210794534
Hamming Loss:  0.22242297892054663
 
Iteration: 3
Best k: 4
Majority Label for cluster  0 :  ('Leptodactylidae', 'Adenome

### 3.)

In GitHub folder as pdf labeled Homework 7, Question 3

### References

https://stackabuse.com/implementing-svm-and-kernel-svm-with-pythons-scikit-learn/

https://stackoverflow.com/questions/43407896/python-sklearn-non-linear-svm-penalty

https://scikit-learn.org/stable/modules/svm.html

https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-create-a-multilabel-svm-classifier-with-scikit-learn.md

https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html

https://scikit-learn.org/stable/modules/multiclass.html

https://stackoverflow.com/questions/38697982/python-scikit-learn-multi-class-multi-label-performance-metrics

https://www.geeksforgeeks.org/python-find-most-frequent-element-in-a-list/

https://stackoverflow.com/questions/36195457/how-to-get-the-samples-in-each-cluster

https://predictivehacks.com/k-means-elbow-method-code-for-python/

https://stackoverflow.com/questions/15376075/cluster-analysis-in-r-determine-the-optimal-number-of-clusters/15376462

https://stackoverflow.com/questions/8459231/sort-tuples-based-on-second-parameter

https://www.statology.org/hamming-distance-python/